In [27]:
import datetime
HEADER="""INSERT INTO public."User"(
	"Id", "Name", "BirthDate", "Gender", "ValidFrom", "ValidTo", "LocationId", "SegmentId") VALUES
"""
ROW="( {user_id}, '{name}', '{birth_date}','{gender}', '{from_date}', '{to_date}', {location_id}, {segment_id} )"
RECORDS=10
LOCATION_RANGE=(1,17) # 17 locations
SEGMENT_RANGE=(1,3) # 3 segments
FUTURE=datetime.date(2999,12,31)
GENDERS=["M","F","O"]
BIRTH_DATE_START=datetime.date(1950,1,1)
BIRTH_DATE_RANGE=365*55 # 55 years


In [28]:
import faker
import random

faker_name = faker.Faker()
today = datetime.date.today()
insert_users = []
for i in range(1,RECORDS+1):
    name = faker_name.name()
    birth_date = BIRTH_DATE_START + datetime.timedelta(days=random.randint(0,BIRTH_DATE_RANGE))
    from_date = today - datetime.timedelta(days=random.randint(180,365*5))
    location_id = random.randint(*LOCATION_RANGE)
    gender = random.choice(GENDERS)
    segment_id = random.randint(*SEGMENT_RANGE)
    insert_users.append(
        dict(
            user_id=i,
            name= name,
            birth_date= birth_date,
            from_date= from_date,
            to_date= FUTURE,
            gender=gender,
            location_id=location_id,
            segment_id=segment_id
        )
    )
    print(ROW.format(**insert_users[-1]))



( 1, 'David Dougherty', '1962-01-05','M', '2019-04-30', '2999-12-31', 5, 2 )
( 2, 'Cameron Stone', '2003-10-03','M', '2021-02-05', '2999-12-31', 2, 2 )
( 3, 'Larry Meadows', '1979-06-23','F', '2022-07-09', '2999-12-31', 15, 1 )
( 4, 'Tara Henry', '1970-04-09','O', '2023-02-14', '2999-12-31', 4, 3 )
( 5, 'Tracy Kelly', '1964-01-10','O', '2021-06-04', '2999-12-31', 15, 1 )
( 6, 'Jose Stafford', '1951-10-27','O', '2020-05-31', '2999-12-31', 2, 1 )
( 7, 'Ryan West', '1989-12-18','F', '2020-02-14', '2999-12-31', 1, 3 )
( 8, 'Warren Diaz', '1963-01-09','O', '2020-02-20', '2999-12-31', 16, 2 )
( 9, 'Leah Stewart', '1987-10-12','M', '2020-06-12', '2999-12-31', 10, 2 )
( 10, 'Michael Torres', '1997-02-02','F', '2022-02-04', '2999-12-31', 8, 2 )


In [29]:
## MAKE INSERT STATEMENT
insert_sql = f'{HEADER}'
insert_sql += ',\n'.join([ROW.format(**user) for user in insert_users])
insert_sql += ';'
print(insert_sql)


INSERT INTO public."User"(
	"Id", "Name", "BirthDate", "Gender", "ValidFrom", "ValidTo", "LocationId", "SegmentId") VALUES
( 1, 'David Dougherty', '1962-01-05','M', '2019-04-30', '2999-12-31', 5, 2 ),
( 2, 'Cameron Stone', '2003-10-03','M', '2021-02-05', '2999-12-31', 2, 2 ),
( 3, 'Larry Meadows', '1979-06-23','F', '2022-07-09', '2999-12-31', 15, 1 ),
( 4, 'Tara Henry', '1970-04-09','O', '2023-02-14', '2999-12-31', 4, 3 ),
( 5, 'Tracy Kelly', '1964-01-10','O', '2021-06-04', '2999-12-31', 15, 1 ),
( 6, 'Jose Stafford', '1951-10-27','O', '2020-05-31', '2999-12-31', 2, 1 ),
( 7, 'Ryan West', '1989-12-18','F', '2020-02-14', '2999-12-31', 1, 3 ),
( 8, 'Warren Diaz', '1963-01-09','O', '2020-02-20', '2999-12-31', 16, 2 ),
( 9, 'Leah Stewart', '1987-10-12','M', '2020-06-12', '2999-12-31', 10, 2 ),
( 10, 'Michael Torres', '1997-02-02','F', '2022-02-04', '2999-12-31', 8, 2 );
